In [1]:
import tensorflow as tf
from memory import Memory
import numpy as np
import math

In [2]:
batch_size = 2
read_heads = 1

mem_size = 10
vector_size = 5

memory = Memory(mem_size, vector_size, read_heads, batch_size)
memory_state = memory.init_memory()
memory_state

(<tf.Tensor 'Fill:0' shape=(2, 10, 5) dtype=float32>,
 <tf.Tensor 'Fill_1:0' shape=(2, 10, 1) dtype=float32>,
 <tf.Tensor 'Fill_2:0' shape=(2, 10, 1) dtype=float32>,
 <tf.Tensor 'Fill_3:0' shape=(2, 5, 1) dtype=float32>)

In [3]:
keys = tf.fill([batch_size,vector_size,read_heads], 1.0)
print('keys',keys)
strengths = tf.fill([batch_size,read_heads], 1.0)
print('strengths',strengths)
interpolation_gate = tf.fill([batch_size,read_heads], 1.0)
print('gate',interpolation_gate)

#shift_weighting = tf.reshape(tf.constant([0.2, 0.6, 0.2, 0.3, 0.5, 0.1, 0.2, 0.4, 0.1, 0.2, 0.3, 0.25]),(-1,3,read_heads))
shift_weighting = tf.reshape(tf.constant([0.2, 0.4, 0.1, 0.2, 0.3, 0.25]),(-1,3,read_heads))
shift_weighting = tf.nn.softmax(shift_weighting,dim=1)
print('shift',shift_weighting)
gamma = tf.fill([batch_size,read_heads], 1.0)
print('gamma',gamma)

add = tf.constant([[2.0, 1.6, 4.0, 1.0, 0.1],[1.0, 1.1, 2.0, 1.0, 0.1]])
erase = tf.constant([[0.0, 0.0, 0.0, 0.0, 0.0],[0.5, 0.5, 0.2, 0.0, 0.0]])
print('add',add)
print('erase',erase)

('keys', <tf.Tensor 'Fill_4:0' shape=(2, 5, 1) dtype=float32>)
('strengths', <tf.Tensor 'Fill_5:0' shape=(2, 1) dtype=float32>)
('gate', <tf.Tensor 'Fill_6:0' shape=(2, 1) dtype=float32>)
('shift', <tf.Tensor 'transpose_1:0' shape=(2, 3, 1) dtype=float32>)
('gamma', <tf.Tensor 'Fill_7:0' shape=(2, 1) dtype=float32>)
('add', <tf.Tensor 'Const_1:0' shape=(2, 5) dtype=float32>)
('erase', <tf.Tensor 'Const_2:0' shape=(2, 5) dtype=float32>)


## Get content addressing

In [4]:
normalized_memory = tf.nn.l2_normalize(memory_state[0], 2)
normalized_keys = tf.nn.l2_normalize(keys, 1)

similiarity = tf.batch_matmul(normalized_memory, normalized_keys)
print(similiarity)

strengths = tf.expand_dims(strengths, 1)
print(strengths)

content_weights = tf.nn.softmax(similiarity * strengths, 1)
print(content_weights)

Tensor("BatchMatMul:0", shape=(2, 10, 1), dtype=float32)
Tensor("ExpandDims:0", shape=(2, 1, 1), dtype=float32)
Tensor("transpose_3:0", shape=(2, 10, 1), dtype=float32)


## Test Interpolation

In [5]:
interpolation_gate = tf.expand_dims(interpolation_gate,1)
gated_weighting = interpolation_gate * content_weights + (1 - interpolation_gate) * memory_state[2]
print(gated_weighting)

Tensor("add:0", shape=(2, 10, 1), dtype=float32)


## Circular convolution

In [6]:
gated_weighting = tf.constant([[[2.0, 1.6, 4.0, 1.0, 0.1, 0.2, 0.4, 2, 3.5, 10],[1.0, 0.5, 2.0, 1.1, 3.1, 0.25, 0.33, 0.14, 0.1, 0]],
                               [[1.0, 1.6, 3.0, 1.0, 0.21, 0.2, 0.6, 1, 0.5, 1],[0.0, 0.0, 1.0, 0.1, 0.1, 0.15, 0.53, 1, 0.1, 0]]])

gated_weighting = tf.reshape(gated_weighting,(2,10,2))
gated_weighting

<tf.Tensor 'Reshape_5:0' shape=(2, 10, 2) dtype=float32>

In [6]:
size = int(gated_weighting.get_shape()[1])
kernel_size = int(shift_weighting.get_shape()[1])
kernel_shift = int(math.floor(kernel_size/2.0))

def loop(idx):
    if idx < 0: return size + idx
    if idx >= size : return idx - size
    else: return idx

kernels = []
for i in xrange(size):
    indices = [loop(i+j) for j in xrange(kernel_shift, -kernel_shift-1, -1)]
    v_ = tf.transpose(tf.gather(tf.transpose(gated_weighting,perm=(1,2,0)), indices),perm=(2,0,1))
    kernels.append(tf.reduce_sum(v_ * shift_weighting, 1))

result_after_shift = tf.stack(kernels, axis=1)
result_after_shift

<tf.Tensor 'stack:0' shape=(2, 10, 1) dtype=float32>

## Sharpening

In [7]:
powed_conv_w = tf.pow(result_after_shift, tf.expand_dims(gamma,1))
after_sharp = powed_conv_w / tf.expand_dims(tf.reduce_sum(powed_conv_w,1),1)
after_sharp

<tf.Tensor 'div:0' shape=(2, 10, 1) dtype=float32>

## Update memory

In [8]:
write_vector = tf.expand_dims(add, 1)
erase_vector = tf.expand_dims(erase, 1)
erasing = memory_state[0] * (1 - tf.batch_matmul(after_sharp, erase_vector))
writing = tf.batch_matmul(after_sharp, write_vector)
updated_memory = erasing + writing
print(updated_memory)

Tensor("add_1:0", shape=(2, 10, 5), dtype=float32)


In [4]:
new_weigth, new_memory = memory.write(memory_state[0],memory_state[1],keys,strengths,interpolation_gate,shift_weighting,gamma,add,erase)
print(new_weigth)
print(new_memory)

Tensor("div:0", shape=(2, 10, 1), dtype=float32)
Tensor("add_1:0", shape=(2, 10, 5), dtype=float32)


## OR read memory

In [83]:
updated_read_vectors = tf.batch_matmul(memory_state[0], after_sharp, adj_x=True)
updated_read_vectors

<tf.Tensor 'BatchMatMul_6:0' shape=(2, 5, 2) dtype=float32>

In [5]:
new_read_weigth, new_read_vector = memory.read(memory_state[0],memory_state[2],keys,strengths,interpolation_gate,shift_weighting,gamma)
print(new_read_weigth)
print(new_read_vector)

Tensor("div_1:0", shape=(2, 10, 2), dtype=float32)
Tensor("BatchMatMul_3:0", shape=(2, 5, 2), dtype=float32)


In [85]:
with tf.Session() as session:
    values = session.run(v_)
    print(values)

[[[ 2.          1.60000002]
  [ 0.1         0.        ]
  [ 0.33000001  0.14      ]]

 [[ 1.          1.60000002]
  [ 0.1         0.        ]
  [ 0.52999997  1.        ]]]
